In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

C:\Users\kevin\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# Read csv
df_data = pd.read_csv('E:\OneDrive\Arbeit\Repos\DMC2022\Kevin\csv\complete_dataset_cleaned_labeled.csv', sep='|')
#df_data.head()

In [ ]:
#df_data.info()

In [ ]:
# Convert categories to data type category
#df_data["categories"] = df_data["categories"].astype("category")

In [ ]:
#df_data.info()

# Preprocessing

In [3]:
# Drop 'Unnamed: 0' column
df_data.drop('Unnamed: 0', axis=1, inplace=True)

### Split datetime into separate columns

In [4]:
"""
# Convert 'date' to datetime
df_data['date'] = pd.to_datetime(df_data['date'], format="%Y-%m-%d")
"""

In [ ]:
"""
# Split 'date' into seperate columns
df_data['day'] = df_data['date'].dt.day
df_data['month'] = df_data['date'].dt.month
df_data['year'] = df_data['date'].dt.year
"""

In [ ]:
"""
# Move new columns to the front
col = df_data.pop("year")
df_data.insert(1, col.name, col)

col = df_data.pop("month")
df_data.insert(2, col.name, col)

col = df_data.pop("day")
df_data.insert(3, col.name, col)
"""

In [5]:
# Drop date column since it's no longer needed
df_data.drop('date', axis=1, inplace=True)

In [ ]:
# Show table
#df_data.head()

### Multi-Hot-Encoding for categories

In [ ]:
"""
# If you run out of memory while encoding the whole dataframe at once you can 
# split the dataframe into chunks beforehand 

df_chunk1 = df.iloc[:500000,:]
df_chunk2 = df.iloc[500000:,:]

"""

In [6]:
# WARNING: This Cell takes a couple of minutes to compute; ~50GB RAM needed to compute without chunks

# Convert categorical variable into dummy/indicator variables
df_multi_hot = df_data["categories"].str.get_dummies(',')

In [7]:
# Replace symbols in columns
df_multi_hot.columns = df_multi_hot.columns.str.replace("[", "")
df_multi_hot.columns = df_multi_hot.columns.str.replace("]", "")

# Combine df_data and df_multi_hot
df_combined = df_data.join(df_multi_hot, how='outer')

# delete df_multi_hot to free up RAM
del df_multi_hot

C:\Users\kevin\AppData\Local\Temp\ipykernel_4256\1813249752.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_multi_hot.columns = df_multi_hot.columns.str.replace("[", "")
C:\Users\kevin\AppData\Local\Temp\ipykernel_4256\1813249752.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_multi_hot.columns = df_multi_hot.columns.str.replace("]", "")


In [ ]:
df_combined.info()

In [12]:
df_combined_chunk1 = df_combined.iloc[:500000,:]
col = df_combined_chunk1.pop("week")
df_combined_chunk1.insert(6266, col.name, col)

#df_combined_chunk2 = df_combined.iloc[500000:,:]

KeyboardInterrupt: 

In [14]:
del df_combined_chunk1

In [ ]:
import gc

gc.collect()

In [ ]:
col = df_combined_chunk1.pop("week")
df_combined_chunk1.insert(6270, col.name, col)

In [ ]:
# Check if we have any missing values
df_combined[df_combined.isnull().any(axis=1)]

In [ ]:
"""
For Testing purposes

# Read csv
df_cat = pd.read_csv('E:\OneDrive\Arbeit\Repos\DMC2022\Kevin\csv\complete_dataset_cleaned_labeled.csv', sep='|')
df_cat["categories"] = df_cat["categories"].astype("string")

df_cat.drop('Unnamed: 0', axis=1, inplace=True)
df_cat.drop('date', axis=1, inplace=True)
df_cat.drop('userID', axis=1, inplace=True)
df_cat.drop('itemID', axis=1, inplace=True)
df_cat.drop('order', axis=1, inplace=True)
df_cat.drop('brand', axis=1, inplace=True)
df_cat.drop('feature_1', axis=1, inplace=True)
df_cat.drop('feature_2', axis=1, inplace=True)
df_cat.drop('feature_3', axis=1, inplace=True)
df_cat.drop('feature_4', axis=1, inplace=True)
df_cat.drop('feature_5', axis=1, inplace=True)
df_cat.drop('week', axis=1, inplace=True)

df_cat = df_cat.head(3)
df_cat
"""

### Datatypes for XGBoost

XGBoost natively supports continuous data but not categorical data. In order to use categorical data with XGBoost, we have to use One-Hot-Encoding which converts a column of categorical values into muliple columns of binary values.

# Modeling

In [ ]:
df_data.drop('categories', axis=1, inplace=True)

In [ ]:
df_data.info()

In [ ]:
df_data.head()

In [ ]:
df_data.drop('order', axis=1, inplace=True)

In [ ]:
# Comma is being used to extract a specific column from a 2D array.
# X = data.iloc[:,:-1]
# X = all rows, all columns except the last one 

X = df_data.iloc[:,3:-1]
X

In [ ]:
y = df_data.iloc[:,-1]
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
data_dmatrix = xgb.DMatrix(data=X, label=y)

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
model.score(X,y)

In [ ]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#model = XGBClassifier(tree_method='gpu_hist', gpu_id=0)
#model.fit(X_train, y_train)

In [ ]:
#xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
#xg_reg.fit(X_train,y_train)

#preds = xg_reg.predict(X_test)

In [ ]:
# Je größer der RMSE ist, desto schlechter ist die Anpassung des Modells.
#rmse = np.sqrt(mean_squared_error(y_test, preds))
#print("RMSE: %f" % (rmse))

In [ ]:
#!conda list